In [2]:
from deltalake import DeltaTable
import numpy as np
import pandas as pd

bets = DeltaTable("/Users/andreikirpichev/Downloads/bets/")
bets = bets.to_pandas()
bets.head()
bets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4871648 entries, 0 to 4871647
Data columns (total 23 columns):
 #   Column                Dtype         
---  ------                -----         
 0   customer_id           object        
 1   age                   int64         
 2   country               object        
 3   section               object        
 4   point                 object        
 5   market                object        
 6   market_params         object        
 7   contest_id            object        
 8   placement_date        object        
 9   placement_time        datetime64[ns]
 10  placement_odds        float32       
 11  home                  object        
 12  away                  object        
 13  region                object        
 14  sport                 object        
 15  league                object        
 16  minutes_before_start  float64       
 17  stake_eur             float64       
 18  dayofweek             int32         
 19  

In [15]:
from sklearn.preprocessing import LabelEncoder

categorical_features = ['country', 'section', 'point', 'market', 'market_params', 'contest_id', 'home', 'away', 'region', 'sport', 'league']

le = LabelEncoder()

# Convert the variables to numerical
for i in range(len(categorical_features)):
    label = le.fit_transform(bets[categorical_features[i]])
    bets[categorical_features[i]] = label
bets.head()


,customer_id,age,country,section,point,market,market_params,contest_id,placement_date,placement_time,...,region,sport,league,minutes_before_start,stake_eur,dayofweek,weekday_stake,section_2w,point_2w,market_2w
0,9872510,39,4,29,18,3,2320,12572,2022-09-24,2022-09-24 10:17:59.980,...,61,9,1071,0.0,3.8,7,8.52,9.19,9.19,9.19
1,4143910,39,2,37,7,4,2320,28669,2022-09-24,2022-09-24 10:18:06.130,...,68,13,783,0.0,20.5,7,6.32,7.33,6.88,14.23
2,4136610,44,2,29,6,3,2320,68058,2022-09-24,2022-09-24 10:18:08.910,...,130,28,1263,783.0,7.5,7,4.58,7.89,6.25,6.55
3,4136610,44,2,29,21,3,2320,67986,2022-09-24,2022-09-24 10:18:08.910,...,107,28,269,0.0,7.5,7,3.92,7.89,7.91,6.55
4,3749310,67,2,37,7,4,2320,28661,2022-09-24,2022-09-24 10:18:18.830,...,68,13,783,0.0,1.0,7,0.94,0.53,0.69,0.98


In [17]:
#show rows with NaN values
X[X.isna().any(axis=1)]

,customer_id,age,country,section,point,market,market_params,contest_id,home,away,region,sport,league,minutes_before_start,weekday_stake,section_2w,point_2w,market_2w


In [25]:
# Create features and target
X = bets[['customer_id', 'age', 'country', 'section', 'point', 'market', 'market_params', 'contest_id', 'home', 'away', 'region', 'sport', 'league', 'minutes_before_start', 'weekday_stake', 'section_2w', 'point_2w', 'market_2w']]
y = bets[['stake_eur']]

# Make necessary imports
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 101)

# Train the model
regr = RandomForestRegressor(n_estimators = 10, max_depth = 10, random_state = 101)
regr.fit(X_train, y_train.values.ravel())

RandomForestRegressor(max_depth=10, n_estimators=10, random_state=101)

In [26]:
import warnings
warnings.filterwarnings('ignore')

# Make prediction
predictions = regr.predict(X_test)

result = X_test
result['stake_eur'] = y_test
result['prediction'] = predictions.tolist()
result.head()

,customer_id,age,country,section,point,market,market_params,contest_id,home,away,region,sport,league,minutes_before_start,weekday_stake,section_2w,point_2w,market_2w,stake_eur,prediction
1912175,3725610,45,2,37,7,15,2320,16731,19658,6158,33,13,175,1.0,3.53,3.17,3.16,2.36,3.33,3.276333
2679583,3800010,56,2,10,7,20,2320,38814,15446,3323,6,13,1178,123.0,0.41,0.32,0.31,0.23,0.25,0.226091
2330869,15335110,53,2,37,7,12,2320,21588,6167,17147,43,13,1465,83.0,1.24,1.10,1.10,1.19,0.35,1.174055
2262266,7920610,21,2,37,7,4,2320,21013,11105,330,43,13,1465,9.0,1.33,1.16,1.17,1.31,4.00,1.368346
3504300,4567110,38,2,37,7,12,2320,57310,4683,5066,43,13,1471,19.0,4.09,4.11,4.14,4.15,2.50,4.322797


In [20]:
# Import library for metrics
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Mean absolute error (MAE)
mae = mean_absolute_error(y_test.values.ravel(), predictions)

# Mean squared error (MSE)
mse = mean_squared_error(y_test.values.ravel(), predictions)

# R-squared scores
r2 = r2_score(y_test.values.ravel(), predictions)

# Print metrics
print('Mean Absolute Error:', round(mae, 2))
print('Mean Squared Error:', round(mse, 2))
print('R-squared scores:', round(r2, 2))

TypeError: Invalid argument, not a string or column: 5.132292782271063 of type <class 'numpy.float64'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.